# Gemini Thinking + Tool Call Test

Gemini 3 Flash Preview에서 `includeThoughts: true` 설정 시:
- `candidates[0].content.parts`에 `thought: true`인 파트가 포함되는지 확인
- Tool call과 thinking이 함께 오는지 확인
- `raw_response` 구조 검증

In [ ]:
import os
from google import genai
from google.genai import types

GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY", "")
if not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY 환경변수를 설정하세요")

client = genai.Client(api_key=GOOGLE_API_KEY)
print("Client ready")

In [ ]:
# Tool definition
move_tool = types.Tool(
    function_declarations=[
        types.FunctionDeclaration(
            name="move",
            description="Move the player in a direction",
            parameters=types.Schema(
                type="OBJECT",
                properties={
                    "direction": types.Schema(type="STRING", enum=["UP", "DOWN", "LEFT", "RIGHT"]),
                    "steps": types.Schema(type="INTEGER"),
                },
                required=["direction", "steps"],
            ),
        )
    ]
)

print("Tool defined:", move_tool.function_declarations[0].name)

## Test 1: Thinking + Tool Call

In [ ]:
# thinking 활성화 + tool call 테스트
MODEL = "gemini-2.5-flash-preview-05-20"

response = client.models.generate_content(
    model=MODEL,
    contents="I'm at position (5, 5) on a 50x50 grid. I need to reach (8, 3). Which direction should I move?",
    config=types.GenerateContentConfig(
        tools=[move_tool],
        thinking_config=types.ThinkingConfig(thinking_budget=1024),
    ),
)

print(f"Model: {MODEL}")
print(f"Candidates: {len(response.candidates)}")
print(f"Parts count: {len(response.candidates[0].content.parts)}")
print()

In [ ]:
# 각 파트 분석
for i, part in enumerate(response.candidates[0].content.parts):
    part_dict = type(part).to_dict(part)
    is_thought = part_dict.get("thought", False)
    has_text = "text" in part_dict
    has_fc = "function_call" in part_dict
    
    print(f"--- Part [{i}] ---")
    print(f"  thought: {is_thought}")
    print(f"  has_text: {has_text}")
    print(f"  has_function_call: {has_fc}")
    
    if is_thought and has_text:
        print(f"  thought_text: {part_dict['text'][:300]}...")
    elif has_fc:
        fc = part_dict["function_call"]
        print(f"  function_call: {fc['name']}({fc.get('args', {})})")
    elif has_text:
        print(f"  text: {part_dict['text'][:200]}")
    print()

## Test 2: Raw response 구조 확인

DataCollector가 저장하는 `raw_response.raw.candidates[0].content.parts` 구조와 동일한지 검증

In [ ]:
import json

# response를 dict로 변환하여 raw 구조 확인
raw_dict = type(response).to_dict(response)

parts = raw_dict["candidates"][0]["content"]["parts"]

# thought 파트 추출 (DataCollector.extractThought와 동일한 로직)
thought_parts = [p for p in parts if p.get("thought") == True]
fc_parts = [p for p in parts if "function_call" in p]

print(f"Total parts: {len(parts)}")
print(f"Thought parts: {len(thought_parts)}")
print(f"FunctionCall parts: {len(fc_parts)}")
print()

if thought_parts:
    thought_text = "\n".join(p["text"] for p in thought_parts)
    print(f"=== Extracted Thought ===")
    print(thought_text[:500])
    print()

if fc_parts:
    print(f"=== Function Calls ===")
    for p in fc_parts:
        fc = p["function_call"]
        print(f"  {fc['name']}({json.dumps(fc.get('args', {}))})")

In [ ]:
# usage metadata
usage = raw_dict.get("usage_metadata", {})
print("Usage Metadata:")
print(json.dumps(usage, indent=2))